Preprocessing 

Configure the paths and the libraries used for preprocessing data

In [7]:
import pandas as pd
import os

print(os.getcwd())


c:\Users\Adria Espinoza\FIB\mds\Q2\ML\ml-project\src


Extract data

In [ ]:

df = pd.read_csv("../data/raw_files/train.csv")

print("Dataframe columns")
print(df.columns)

print("Dataframe rows")

DF head:


AttributeError: 'Index' object has no attribute 'index'